In [1]:
import csv
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split  
import string
import re

In [2]:
docs = pd.read_excel("dataset-emosi-twitter-2500.xlsx", engine="openpyxl")
docs.head()

,LABEL,TEKS
0,terkejut,@indomyfess Seokjin\n\nYah pas kenal Bangtan t...
1,jijik,@JaneeeNoLimit Wkwk. Iri dengki dah
2,sedih,Tadi gue lewat jalan yang aga rame orang juala...
3,jijik,@sunrisingbok LWBSKSN yo igual muak
4,terkejut,@moontygf EMG ANJRIT GUA KAGET GWS BGT NAD<U+0...


In [3]:
kelas=docs.LABEL.value_counts()
kelas

terkejut    723
jijik       716
senang      331
marah       308
sedih       280
takut       142
Name: LABEL, dtype: int64

In [4]:
X = docs.TEKS
y = docs.LABEL
print(X.shape)
print(y.shape)

(2500,)
(2500,)


In [5]:
def cleaning(data):
    text=[]
    
    tweet= data.lower()
    
         # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
        # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
        # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
        # remove hashtags
        # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
        #remove @
    tweet = re.sub(r"@\S+", " ", tweet)
    
    tweet = re.sub("[^a-zA-Z]", " ",tweet)
    
        
    text.append(tweet)
    
    return text

In [6]:
def slangword(tweet):
    convert = pd.read_csv('dataslang.csv', sep=',' ,header=None, names=['slang', 'fix'])
    slangword= []
    slang=[]
    
    for i in (convert['slang']):
        slangword.append(i)

   
   
    i=tweet.split()
    
    temp = ''

    for j in i:
        if (j in slangword):
            for c in range(len(convert)):
                if(convert['slang'][c] == j):
                    hasil = convert['fix'][c]
                    temp= temp + ' ' + hasil
                else:
                    temp = temp
                
        else:       
            temp= temp + ' '+ j
                
    slang.append(temp)
    return slang

In [7]:
def negation(kotak):
    negasi = ['tidak', 'bukan']
    wadah = []

    for i in kotak:
        
        i=i.split()
        n= 0
        k=0
        temp=''
        p = 0
        
        for j in i:
            p+=1
           
            if j in negasi and p != len(i):
                n= n+1
                k= n                
                hasil = j + i[n]
                temp= temp + ' '+ hasil            
            elif j == i[k] and n!= 0 :
                n= n+1
                temp= temp                
            else:
                n= n+1
                temp= temp +' '+ j
         
        wadah.append(temp)
    
    return wadah


In [8]:
def gabung (tweet):
    text=[]
    for j in tweet:
        spasi = ''
        for i in j:
            spasi = spasi + ' ' + i
        text.append(spasi)
    return text

In [9]:
def clear(data):
    wadah=[]
    for i in data:
        clean = cleaning(i)
        #print(clean)
        #slang = slangword(clean)
        #print(slang)
        #negasi = negation(slang)
        #print(negasi)
        stop = stopword(clean)
        #print(stop)
        wadah.append(stop)
    #print(wadah)
    return wadah

In [10]:
def preprocessing(wadah):
    data= clear(wadah)
    #print(data)
    hasil = gabung(data)
    #print(hasil)
   
    return hasil

In [11]:
def stopword(wadah):
    
    factory = StopWordRemoverFactory()
    more_stopword = ['dengan', 'ia','bahwa','oleh','aku','kamu','bukan']
    data =factory.get_stop_words()+more_stopword
    stopword = factory.create_stop_word_remover()
    stemmer = StemmerFactory()
    stemmer = stemmer.create_stemmer()
    # Kalimat
    
    for i in wadah:
        stop=(word_tokenize(stemmer.stem(stopword.remove(i))))
    return stop

In [12]:
X_train = preprocessing(X)

In [13]:
print(X_train)

[' seokjin yah pas kenal bangtan tiba lihat seokjin jadi pesona awowkwk ke semua jg sih', ' wkwk iri dengki dah', ' tadi gue lewat jalan aga rame orang jual gua liat yg dagang baju baju tulis gantipresiden gila mesti ga sih sampe kaya gitunya kadang gua bingung ama orang indonesia ga pernah rasa syukur', ' lwbsksn yo igual muak', ' emg anjrit gua kaget gws bgt nad u f f u f f', ' laku cermin laku pasang kalo nyebelin siap aja punya pasang yg nyebelin teman q kenapa sih apa', ' username username promosi rencana baik sempat minta utk libat buat program venue palembang siap program survey nyata batal alas dana jadi turun kasar nafsu besar tenag', ' relate banget kdng bbrp sampe gue capek ngingetinnya bc ulang trs smpe bikin w muak bkn cuma cara ngedidik ortu anak sikap sama tindak bikin w hela napas tpi ga gue pungkiri kok dik yg bagus yaa gitu nama manusia', ' spelah dengki op ni', ' jangan iri jangan iri jangan iri dengki jangan jangan iri sakit hati u e u e u e u e u e u e u e u e u e 

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mutual_info_score
import math


In [15]:
def count_word_in_class(x,y):
    count = {}
    count_word = {}
    word_class = {}
    for i in range(0, len(x)):
        x_split = x[i].split()
        for word in x_split:
            w_c = word+str(",")+str(y[i])
            if (w_c in count):
                count[w_c] += 1
            else:
                count[w_c] = 1
                
            if (word in count_word):
                count_word[word] += 1
            else:
                count_word[word] = 1
                
            if (y[i] in word_class):
                word_class[y[i]] += 1
            else:
                word_class[y[i]] = 1
    return count, count_word, word_class

In [16]:
def count_class_in_doc(y):
    count = {}
    for i in range(0, len(y)):
        if (y[i] in count):
            count[y[i]] += 1
        else:
            count[y[i]] = 1
    return count

In [17]:
def hitung_prob(count,word,uniq):
    
    return ((count+1)/(word+uniq))

In [18]:
def multiNB(x,y,x_test):
    count_word_on_class, count_word, word_class = count_word_in_class(x,y)

    count_class = count_class_in_doc(y)

    kelaz = list(count_class.keys())
    y_pred = []
    for i in range(0,len(x_test)):
        prob_kelas = {}
        for j in count_class.keys():
            prob_c = 1
            x_split = x_test[i].split()
            for k in x_split:
                tag = k+str(',')+str(j)
                prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
            nama = str(i)+','+str(j)
            prob_kelas[nama] = prob_c
        y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    return y_pred

In [19]:
def get_subset(dataSet, indexes):
    """
    Selects subset of data based on indexes given
    from KFold validation method
    Returns:
        Subset of data set
    """
    return [dataSet[i] for i in indexes]

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn import metrics
kf = KFold(n_splits=10, shuffle=True)
sumakurasi = 0
for train_index, test_index in kf.split(X):
    features_train = get_subset(X_train, train_index)
    labels_train = get_subset(y, train_index)
    features_test = get_subset(X_train, test_index)
    labels_test = get_subset(y, test_index)
    
    akurasi = []
    
    y_pred_nb = multiNB(features_train, labels_train,  features_test)
    acc_nb = accuracy_score(labels_test,  y_pred_nb)*100
    sumakurasi += acc_nb
    report_nb = classification_report(labels_test, y_pred_nb)
    akurasi.append(acc_nb)
    print('++++')
    #print(y_pred_nb)
    print(acc_nb)
    
    print("Report 1: Naive Bayes")
    print(report_nb)
    
print('====')
print('rata akurasi: ' ,(sumakurasi/10))

++++
71.2
Report 1: Naive Bayes
              precision    recall  f1-score   support

       jijik       0.88      0.96      0.92        74
       marah       0.39      0.30      0.34        30
       sedih       0.42      0.57      0.48        28
      senang       0.54      0.38      0.45        39
       takut       0.64      0.82      0.72        11
    terkejut       0.88      0.85      0.87        68

    accuracy                           0.71       250
   macro avg       0.62      0.65      0.63       250
weighted avg       0.70      0.71      0.70       250

++++
70.0
Report 1: Naive Bayes
              precision    recall  f1-score   support

       jijik       0.92      0.92      0.92        73
       marah       0.46      0.41      0.43        32
       sedih       0.20      0.29      0.24        24
      senang       0.48      0.33      0.39        36
       takut       0.83      0.67      0.74        15
    terkejut       0.86      0.94      0.90        70

    accuracy 